# Harmonize emissions

Take the pyam output and make the pathways consistent with SSP historical data.

In [1]:
import datetime
import warnings

import aneris.convenience
import pandas as pd
import scmdata
from tqdm.auto import tqdm

/Users/chris/anaconda3/envs/methane-policy-iiasa/lib/python3.10/site-packages/scmdata/database/_database.py:9: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  import tqdm.autonotebook as tqdman


I should have already pre-processed both the RCMIP historical and the GAINS scenarios to have the same variable names and units, which will make things much less painful.

Obviously, CO2 and N2O need to be re-converted.

In [2]:
variables = [
    "Emissions|CO2|Energy and Industrial Processes",
    "Emissions|CO2|AFOLU",
    "Emissions|CH4",
    "Emissions|N2O",
    "Emissions|Sulfur",
    "Emissions|CO",
    "Emissions|VOC",
    "Emissions|NOx",
    "Emissions|BC",
    "Emissions|OC",
    "Emissions|NH3"
]

In [3]:
var_units = {
    "Emissions|CO2|Energy and Industrial Processes": "Mt CO2/yr",
    "Emissions|CO2|AFOLU": "Mt CO2/yr",
    "Emissions|CH4": "Mt CH4/yr",
    "Emissions|N2O": "kt N2O/yr",
    "Emissions|Sulfur": "Mt SO2/yr",
    "Emissions|CO": "Mt CO/yr",
    "Emissions|VOC": "Mt VOC/yr",
    "Emissions|NOx": "Mt NO2/yr",
    "Emissions|BC": "Mt BC/yr",
    "Emissions|OC": "Mt OC/yr",
    "Emissions|NH3": "Mt NH3/yr"
}

In [4]:
times = []
years = range(1750, 2015)
for year in years:
    times.append(datetime.datetime(year, 1, 1))
    # emissions are really midyears, but we just want this to work

In [5]:
history = (
    scmdata.ScmRun(
        "../data/rcmip5.1.0_historical.csv", lowercase_cols=True
    )
    .filter(region="World", variable=variables)
    .interpolate(target_times=times)
    .timeseries(time_axis="year")
)

In [6]:
times_future = []
years_future = range(2014, 2101)
for year in years_future:
    times_future.append(datetime.datetime(year, 1, 1))

In [7]:
future = (
    scmdata.ScmRun("../data/gains_scenarios_pyam_format.csv", lowercase_cols=True)
#    .filter(scenario=scenarios, variable=rcmip_variables, region="World")
    .interpolate(times_future)
    .timeseries(time_axis="year")
)

In [8]:
overrides = pd.DataFrame(
    [
        {
            "method": "reduce_ratio_2150_cov",
            "variable": "Emissions|CO",
        },
        {
            "method": "reduce_ratio_2080",
            "variable": "Emissions|CO2|Energy and Industrial Processes",
        },
        {
            "method": "reduce_offset_2150_cov",
            "variable": "Emissions|CO2|AFOLU",
        },
        {
            "method": "reduce_ratio_2150_cov",
            "variable": "Emissions|OC",
        },
        {
            "method": "reduce_ratio_2150_cov",
            "variable": "Emissions|VOC",
        },
    ]
)

harmonisation_year = 2014

In [9]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    scenarios_harmonised = [
        aneris.convenience.harmonise_all(
            msdf,
            history=history,
            harmonisation_year=harmonisation_year,
            overrides=overrides,
        )
        for _, msdf in tqdm(future.groupby(["model", "scenario"]))
    ]

scenarios_harmonised = pd.concat(scenarios_harmonised).reset_index()

  0%|          | 0/678 [00:00<?, ?it/s]

In [10]:
scenarios_harmonised.to_csv(
    f"../data/gains_scenarios_harmonized.csv", index=False
)